In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
params = np.load('vgg-weights/weights.dat', encoding='bytes')

In [3]:
    params = np.reshape(params, (1,))[0]
    params.keys()
    for key in params.keys():
        params[key] = { k.decode('utf-8') : v for k, v in params[key].items() }

In [7]:
print(params.keys())
print(params['fc8']['weights'].shape)
print(params['fc8']['biases'].shape)

dict_keys(['conv5_1', 'fc6', 'conv5_3', 'conv5_2', 'fc8', 'fc7', 'conv4_1', 'conv4_2', 'conv4_3', 'conv3_3', 'conv3_2', 'conv3_1', 'conv1_1', 'conv1_2', 'conv2_2', 'conv2_1'])
(4096, 1000)
(1000,)


In [7]:
print(params['fc8']['weights'].shape)

(4096, 1000)


In [ ]:
params['conv5_1']['biases']

In [ ]:
# Testing the conv2d operation in TF
inp = tf.placeholder(dtype=tf.float32, shape=(None, 3, 3, 2))
f1 = np.array([[[1,0], [0,1]], [[0,1], [1,0]]], dtype=np.float32)
f2 = np.array([[[-1, 0], [0, -1]], [[0, 1], [1, 0]]], dtype=np.float32)
f3 = np.array([[[-1, 1], [1, -1]], [[-1, 1], [1, -1]]], dtype=np.float32)
f = np.concatenate((np.expand_dims(f1, axis=3), np.expand_dims(f2, axis=3), np.expand_dims(f3, axis=3)), axis=3)
print(f.shape)

In [ ]:
filters = tf.Variable(f, name='filters')
out = tf.nn.conv2d(inp, filters, [1,1,1,1], 'VALID')

In [ ]:
x = np.array([[[[1,2,3],[4,5,6],[7,8,9]], [[7,8,9],[4,5,6],[1,2,3]]]])
y = np.array([[[[1,7],[2,8],[3,9]],[[4,4],[5,5],[6,6]],[[7,1],[8,2],[9,3]]],
              [[[1,7],[2,8],[3,9]],[[4,4],[5,5],[6,6]],[[7,1],[8,2],[9,3]]]])
# y = np.reshape(y, (1, 3, 3, 2))
print(x.shape)
print(y.shape)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    _, out1 = sess.run([inp, out], feed_dict = {inp: y })
    print(out1)
    print(out1.shape)

In [ ]:
params['conv1_1']['weights'].shape

In [ ]:
params['conv1_2']['weights'].shape

In [ ]:
def voc_label_color_map(N=256, normalized=False):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])

    cmap = cmap/255 if normalized else cmap
    return cmap

In [ ]:
def vgg16_model(batch_sz):
    global params
    net = {}

    net['input'] = tf.placeholder(dtype=tf.float32)

    conv1_1_weights = tf.Variable(params['conv1_1']['weights'], name='conv1_1_weights')
    conv1_1_biases = tf.Variable(params['conv1_1']['biases'], name='conv1_1_biases')
    net['conv1_1'] = tf.nn.bias_add(tf.nn.conv2d(net['input'], conv1_1_weights, [1,1,1,1], 'SAME'), conv1_1_biases)
    net['relu1_1'] = tf.nn.relu(net['conv1_1'])

    conv1_2_weights = tf.Variable(params['conv1_2']['weights'], name='conv1_2_weights')
    conv1_2_biases = tf.Variable(params['conv1_2']['biases'], name='conv1_2_biases')
    net['conv1_2'] = tf.nn.bias_add(tf.nn.conv2d(net['relu1_1'], conv1_2_weights, [1,1,1,1], 'SAME'), conv1_2_biases)
    net['relu1_2'] = tf.nn.relu(net['conv1_2'])
    
    net['pool1'] = tf.nn.max_pool(net['relu1_2'], [1,2,2,1], [1,2,2,1], 'SAME')
    
    conv2_1_weights = tf.Variable(params['conv2_1']['weights'], name='conv2_1_weights')
    conv2_1_biases = tf.Variable(params['conv2_1']['biases'], name='conv2_1_biases')
    net['conv2_1'] = tf.nn.bias_add(tf.nn.conv2d(net['pool1'], conv2_1_weights, [1,1,1,1], 'SAME'), conv2_1_biases)
    net['relu2_1'] = tf.nn.relu(net['conv2_1'])
    
    conv2_2_weights = tf.Variable(params['conv2_2']['weights'], name='conv2_2_weights')
    conv2_2_biases = tf.Variable(params['conv2_2']['biases'], name='conv2_2_biases')
    net['conv2_2'] = tf.nn.bias_add(tf.nn.conv2d(net['relu2_1'], conv2_2_weights, [1,1,1,1], 'SAME'), conv2_2_biases)
    net['relu2_2'] = tf.nn.relu(net['conv2_2'])
    
    net['pool2'] = tf.nn.max_pool(net['relu2_2'], [1,2,2,1], [1,2,2,1], 'SAME')
    
    conv3_1_weights = tf.Variable(params['conv3_1']['weights'], name='conv3_1_weights')
    conv3_1_biases = tf.Variable(params['conv3_1']['biases'], name='conv3_1_biases')
    net['conv3_1'] = tf.nn.bias_add(tf.nn.conv2d(net['pool2'], conv3_1_weights, [1,1,1,1], 'SAME'), conv3_1_biases)
    net['relu3_1'] = tf.nn.relu(net['conv3_1'])
    
    conv3_2_weights = tf.Variable(params['conv3_2']['weights'], name='conv3_2_weights')
    conv3_2_biases = tf.Variable(params['conv3_2']['biases'], name='conv3_2_biases')
    net['conv3_2'] = tf.nn.bias_add(tf.nn.conv2d(net['relu3_1'], conv3_2_weights, [1,1,1,1], 'SAME'), conv3_2_biases)
    net['relu3_2'] = tf.nn.relu(net['conv3_2'])
    
    conv3_3_weights = tf.Variable(params['conv3_3']['weights'], name='conv3_3_weights')
    conv3_3_biases = tf.Variable(params['conv3_3']['biases'], name='conv3_3_biases')
    net['conv3_3'] = tf.nn.bias_add(tf.nn.conv2d(net['relu3_2'], conv3_3_weights, [1,1,1,1], 'SAME'), conv3_3_biases)
    net['relu3_3'] = tf.nn.relu(net['conv3_3'])
    
    net['pool3'] = tf.nn.max_pool(net['relu3_3'], [1,2,2,1], [1,2,2,1], 'SAME')
    
    conv4_1_weights = tf.Variable(params['conv4_1']['weights'], name='conv4_1_weights')
    conv4_1_biases = tf.Variable(params['conv4_1']['biases'], name='conv4_1_biases')
    net['conv4_1'] = tf.nn.bias_add(tf.nn.conv2d(net['pool3'], conv4_1_weights, [1,1,1,1], 'SAME'), conv4_1_biases)
    net['relu4_1'] = tf.nn.relu(net['conv4_1'])
    
    conv4_2_weights = tf.Variable(params['conv4_2']['weights'], name='conv4_2_weights')
    conv4_2_biases = tf.Variable(params['conv4_2']['biases'], name='conv4_2_biases')
    net['conv4_2'] = tf.nn.bias_add(tf.nn.conv2d(net['relu4_1'], conv4_2_weights, [1,1,1,1], 'SAME'), conv4_2_biases)
    net['relu4_2'] = tf.nn.relu(net['conv4_2'])
    
    conv4_3_weights = tf.Variable(params['conv4_3']['weights'], name='conv4_3_weights')
    conv4_3_biases = tf.Variable(params['conv4_3']['biases'], name='conv4_3_biases')
    net['conv4_3'] = tf.nn.bias_add(tf.nn.conv2d(net['relu4_2'], conv4_3_weights, [1,1,1,1], 'SAME'), conv4_3_biases)
    net['relu4_3'] = tf.nn.relu(net['conv4_3'])
    
    net['pool4'] = tf.nn.max_pool(net['relu4_3'], [1,2,2,1], [1,2,2,1], 'SAME')
    
    conv5_1_weights = tf.Variable(params['conv5_1']['weights'], name='conv5_1_weights')
    conv5_1_biases = tf.Variable(params['conv5_1']['biases'], name='conv5_1_biases')
    net['conv5_1'] = tf.nn.bias_add(tf.nn.conv2d(net['pool4'], conv5_1_weights, [1,1,1,1], 'SAME'), conv5_1_biases)
    net['relu5_1'] = tf.nn.relu(net['conv5_1'])
    
    conv5_2_weights = tf.Variable(params['conv5_2']['weights'], name='conv5_2_weights')
    conv5_2_biases = tf.Variable(params['conv5_2']['biases'], name='conv5_2_biases')
    net['conv5_2'] = tf.nn.bias_add(tf.nn.conv2d(net['relu5_1'], conv5_2_weights, [1,1,1,1], 'SAME'), conv5_2_biases)
    net['relu5_2'] = tf.nn.relu(net['conv5_2'])
    
    conv5_3_weights = tf.Variable(params['conv5_3']['weights'], name='conv5_3_weights')
    conv5_3_biases = tf.Variable(params['conv5_3']['biases'], name='conv5_3_biases')
    net['conv5_3'] = tf.nn.bias_add(tf.nn.conv2d(net['relu5_2'], conv5_3_weights, [1,1,1,1], 'SAME'), conv5_3_biases)
    net['relu5_3'] = tf.nn.relu(net['conv5_3'])
    
    net['pool5'] = tf.nn.max_pool(net['relu5_3'], [1,2,2,1], [1,2,2,1], 'SAME')
    
    conv6_weights = tf.Variable(tf.truncated_normal([7, 7, 512, 4096]))
    conv6_biases = tf.Variable(tf.zeros([4096,], tf.float32))
    net['conv6'] = tf.nn.bias_add(tf.nn.conv2d(net['pool5'], conv6_weights, [1,1,1,1], 'SAME'), conv6_biases)
    net['relu6'] = tf.nn.relu(net['conv6'])
    
    conv7_weights = tf.Variable(tf.truncated_normal([1, 1, 4096, 4096]))
    conv7_biases = tf.Variable(tf.zeros([4096,], tf.float32))
    net['conv7'] = tf.nn.bias_add(tf.nn.conv2d(net['relu6'], conv7_weights, [1,1,1,1], 'SAME'), conv7_biases)
    net['relu7'] = tf.nn.relu(net['conv7'])
    
    conv8_weights = tf.Variable(tf.truncated_normal([1, 1, 4096, 1000]))
    conv8_biases = tf.Variable(tf.zeros([1000,], tf.float32))
    net['conv8'] = tf.nn.bias_add(tf.nn.conv2d(net['relu7'], conv8_weights, [1, 1, 1, 1], 'SAME'), conv8_biases)
    net['relu8'] = tf.nn.relu(net['conv8'])

    upscore2_weights = tf.Variable(tf.truncated_normal([4, 4, 512, 1000]))
    upscore2_biases = tf.Variable(tf.zeros([512,], tf.float32))
    net['upscore2'] = tf.nn.bias_add(
        tf.nn.conv2d_transpose(net['relu8'], upscore2_weights, [batch_sz, 14, 14, 512], strides=[1, 2, 2, 1], padding='SAME'),
        upscore2_biases
    )
    net['fuse_pool4'] = tf.add(net['upscore2'], net['pool4'])
    
    upscore4_weights = tf.Variable(tf.truncated_normal([4, 4, 256, 512]))
    upscore4_biases = tf.Variable(tf.zeros([256,], tf.float32))
    net['upscore4'] = tf.nn.bias_add(
        tf.nn.conv2d_transpose(net['fuse_pool4'], upscore4_weights, [batch_sz, 28, 28, 256], strides=[1, 2, 2, 1], padding='SAME'),
        upscore4_biases
    )
    net['fuse_pool3'] = tf.add(net['upscore4'], net['pool3'])
    
    upscore32_weights = tf.Variable(tf.truncated_normal([4, 4, 21, 256]))
    upscore32_biases = tf.Variable(tf.zeros([21,], tf.float32))
    net['upscore32'] = tf.nn.bias_add(
        tf.nn.conv2d_transpose(net['fuse_pool3'], upscore32_weights, [batch_sz, 224, 224, 21], strides=[1, 8, 8, 1], padding='SAME'),
        upscore32_biases
    )

    return net

In [ ]:
net = vgg16_model(1)

In [ ]:
net['conv1_1']

In [ ]:
import cv2

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    op, inp = sess.run([net['upscore32'], net['input']], feed_dict = {net['input']: np.array([cv2.resize(cv2.imread('2008_004512.jpg'), (224, 224))])})
    print(op.shape)
    print(inp.shape)
    if op.shape[3] == 64:
        plt.figure(figsize=(20,20))
    elif op.shape[3] == 128:
        plt.figure(figsize=(40,20))
    elif op.shape[3] == 256:
        plt.figure(figsize=(13,13))
    elif op.shape[3] == 512:
        plt.figure(figsize=(20,10))    
    for fidx in range(op.shape[3]):
        if op.shape[3] == 64:
            plt.subplot(8, 8, fidx+1)
        elif op.shape[3] == 128:
            plt.subplot(16, 8, fidx+1)
        elif op.shape[3] == 256:
            plt.subplot(16, 16, fidx+1)
        elif op.shape[3] == 512:
            plt.subplot(32, 16, fidx+1)
        plt.axis('off')
        plt.imshow(op[0][:,:,fidx])
    plt.figure()
    plt.axis('off')
    plt.imshow(op[0][:,:,0])
    plt.imsave(fname='2008_004512_output.png', arr=op[0][:,:,0], format='png')
    plt.figure()
    plt.axis('off')
    plt.imshow(cv2.imread('2008_004512.jpg')[:,:,::-1])
    

In [ ]:

print(params['conv5_3']['weights'].shape)